# JAMS-Test: evaluation of the jamifier

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "choco"))


# Choose plotting style
# sns.set_style("whitegrid")
# sns.set_style("darkgrid")

In [98]:
def preprocess_partition_test_df(partition_test_path:str):

    partition_test_df = pd.read_csv(partition_test_path)
    # Append an extra column with the name of the outer partition
    partition_name = os.path.basename(  # name of double-parent dir
        os.path.dirname(os.path.dirname(partition_test_df["gold"][0])))
    partition_test_df["partition"] = partition_name

    return partition_test_df


def describe_partitions(partition_res_df:pd.DataFrame, dec=4):

    partition_means = partition_res_df.groupby("partition").mean().round(dec)
    partition_stds = partition_res_df.groupby("partition").std().round(dec)
    # String representation of the combined mean ± std partition-wise
    partition_str = partition_means.astype(str) \
        + " ± " + partition_stds.astype(str)

    return partition_means, partition_stds, partition_str


## Processing a single evaluation

In [7]:
test_sample = "../tests/data/dummy-spartition/test/test_results_agg.csv"

test_sample_df = pd.read_csv(test_sample)

test_sample_df

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,gold,jamified,type
0,1.000000,0.900000,1.0,0.58,1.0,1.0,1.0,0.0,0.0,0.0,0.800000,1.000000,0.600000,0.340000,0.400000,0.240000,../tests/data/dummy-spartition/test/score_a_2....,../tests/data/dummy-spartition/jams/score_a.jams,score
1,0.833333,0.694118,1.0,0.80,1.0,1.0,1.0,0.0,0.0,0.0,0.000000,0.833333,0.111111,1.622222,1.444444,0.466667,../tests/data/dummy-spartition/test/score_b_1....,../tests/data/dummy-spartition/jams/score_b.jams,score
2,0.833333,0.694118,1.0,0.80,1.0,1.0,1.0,0.0,0.0,0.0,0.333333,1.000000,0.444444,1.088889,0.888889,0.311111,../tests/data/dummy-spartition/test/score_b_2....,../tests/data/dummy-spartition/jams/score_b.jams,score
3,1.000000,0.973430,1.0,1.00,1.0,1.0,1.0,0.0,0.0,0.0,1.000000,1.000000,0.666667,0.000000,0.000000,0.333333,../tests/data/dummy-spartition/test/score_a_1....,../tests/data/dummy-spartition/jams/score_a.jams,score
4,1.000000,1.000000,1.0,1.00,1.0,1.0,1.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,../tests/data/dummy-spartition/test/score_c_1....,../tests/data/dummy-spartition/jams/score_c.jams,score


## Aggregation of evaluations

In [61]:
test_results = glob.glob("../tests/data/*/test/**/test_results_agg.csv", recursive=True)
print(f"Found {len(test_results)} test result CSV files from partitions")

test_results

Found 2 test result CSV files from partitions


['../tests/data/dummy-apartition/test/test_results_agg.csv',
 '../tests/data/dummy-spartition/test/test_results_agg.csv']

In [92]:
partition_test_dfs = [preprocess_partition_test_df(ptest_path) for ptest_path in test_results]

In [93]:
all_partitions = pd.concat(partition_test_dfs)
# Recompute indexes and drop JAMS-file columns
all_partitions = all_partitions.reset_index(drop=True)
all_partitions = all_partitions.drop(["gold", "jamified"], axis=1)

In [94]:
meta_cols = ["meta_cov", "meta_acc", "iden_cov", "iden_acc", "partition"]
anns_cols = [c for c in all_partitions.columns if c not in meta_cols] + ["partition"]

all_partitions_meta = all_partitions[meta_cols]
all_partitions_anns = all_partitions[anns_cols]

all_partitions_anns_score = all_partitions_anns[all_partitions_anns["type"]=="score"]
all_partitions_anns_audio = all_partitions_anns[all_partitions_anns["type"]=="audio"]

### Metadata results

In [99]:
_, _, meta_res_str = describe_partitions(all_partitions_meta)

In [100]:
meta_res_str

,meta_cov,meta_acc,iden_cov,iden_acc
partition,,,,
dummy-apartition,0.9333 ± 0.0913,0.8523 ± 0.149,1.0 ± 0.0,0.836 ± 0.1746
dummy-spartition,0.9333 ± 0.0913,0.8523 ± 0.149,1.0 ± 0.0,0.836 ± 0.1746


### Annotation results: score

In [101]:
_, _, ann_res_score_str = describe_partitions(all_partitions_anns_score)

In [102]:
ann_res_score_str

,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value
partition,,,,,,,,,,,,
dummy-spartition,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.6267 ± 0.4437,0.9667 ± 0.0745,0.5644 ± 0.3246,0.6102 ± 0.7196,0.5467 ± 0.621,0.2702 ± 0.1719


### Annotation results: audio

In [105]:
_, _, ann_res_audio_str = describe_partitions(all_partitions_anns_audio)

In [106]:
ann_res_audio_str


,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value
partition,,,,,,,,,,,,
dummy-apartition,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.6267 ± 0.4437,0.9667 ± 0.0745,0.5644 ± 0.3246,0.6102 ± 0.7196,0.5467 ± 0.621,0.2702 ± 0.1719


In [110]:
a = ["a", "b", "c"]

b = ["a", "b", "d", "e", "b"]

all([ele in b for ele in a])

False

In [111]:
import jams

jam = jams.JAMS(annotations=[
    jams.Annotation(namespace="chord"),
    jams.Annotation(namespace="chord_harte")
])

In [115]:
jam.search(namespace=r"chord$")

,time,duration,value,confidence


In [128]:
# import jams

# jams.load("/Users/jacopodeberardinis/Documents/projects/choco/partitions/real-book/test/real-book_251_silver_Nicolas.jams", strict=False)

In [131]:
test = pd.DataFrame([
    {
        "name": "a",
        "val_a": 10,
        "val_b": 20,
    },
    {
        "name": "a",
        "val_a": None,
        "val_b": 40,
    },
    {
        "name": "a",
        "val_a": 20,
        "val_b": 20,
    }
])

test = pd.DataFrame([
    {
        "name": "a",
        "val_a": 10,
    },
    {
        "name": "a",
        "val_a": None,
    },
    {
        "name": "a",
        "val_a": 20,
    }
])

In [132]:
test.mean()

/var/folders/q6/phq9pwd56q78175j_yvjz8pc0000gn/T/ipykernel_39112/1706541227.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test.mean()


val_a    15.000000
val_b    26.666667
dtype: float64

In [133]:
a = {"ass": 1}